In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor, Resize, Compose, v2
from torchvision.datasets import Cityscapes
from torch.utils.data import random_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import utils
import wandb



In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: svenbbs. Use `wandb login --relogin` to force relogin


True

In [3]:
lr_rate = 1e-3
epochs = 5
batch_size = 64
subsize = (128, 256)
momentum = 0.9
mean, std = ([0.2776, 0.3131, 0.2740]), ([0.1695, 0.1745, 0.1706])
weights = torch.tensor([3.0492e-04, 7.7058e-04, 1.6697e-03, 1.0166e-03, 1.5975e-03, 8.6582e-04,
        3.7707e-05, 2.2719e-04, 1.2423e-03, 1.9831e-03, 6.5811e-05, 1.0036e-03,
        1.0501e-03, 2.6375e-03, 1.8904e-03, 2.3288e-03, 1.7155e-03, 2.7163e-03,
        1.9230e-03, 1.2427e-03, 9.0243e-05, 9.4631e-04, 3.6473e-04, 1.3061e-03,
        3.7688e-03, 2.2016e-04, 4.6955e-03, 5.7737e-03, 3.3058e-02, 2.2757e-02,
        4.5719e-03, 1.4638e-02, 5.8080e-03, 8.7571e-01]).cuda()

In [4]:
augmented_transform = transforms.Compose([
    v2.ToTensor(),
    v2.Resize(subsize),
    v2.RandomHorizontalFlip(1),  # Random horizontal flip
    #v2.RandomRotation(10),  # Random rotation up to 10 degrees
    #v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random color jitter
    
])

augmented_target_transform = transforms.Compose([
    v2.ToTensor(),
    v2.Resize(subsize),
    v2.RandomHorizontalFlip(1),  # Same transformation as input images
    #v2.RandomRotation(10),  # Same transformation as input images
])

normal_transform = transforms.Compose([
    v2.ToTensor(),
    v2.Resize(subsize),
])
normal_target_transform = transforms.Compose([
    v2.ToTensor(),
    v2.Resize(subsize),
])


#Desktop
normal_dataset = Cityscapes(root="E:\CityScapes", split='train', mode='fine', target_type='semantic', transform=normal_transform, target_transform=normal_target_transform)
augmented_dataset = Cityscapes(root="E:\CityScapes", split='train', mode='fine', target_type='semantic', transform=augmented_transform, target_transform=augmented_target_transform)
dataset = torch.utils.data.ConcatDataset([normal_dataset, augmented_dataset])


#Laptop
#dataset = Cityscapes(root="C:/Users/20182573/Documents/CityScapes", split='train', mode='fine', target_type='semantic', transform=transform, target_transform=target_transform)

#subset_small, subset_big = random_split(dataset, [0.2,0.8])
train_dataset, val_dataset = random_split(dataset, [0.8,0.2])

<>:28: SyntaxWarning: invalid escape sequence '\C'
<>:29: SyntaxWarning: invalid escape sequence '\C'
<>:28: SyntaxWarning: invalid escape sequence '\C'
<>:29: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Sven\AppData\Local\Temp\ipykernel_2132\958200182.py:28: SyntaxWarning: invalid escape sequence '\C'
  normal_dataset = Cityscapes(root="E:\CityScapes", split='train', mode='fine', target_type='semantic', transform=normal_transform, target_transform=normal_target_transform)
C:\Users\Sven\AppData\Local\Temp\ipykernel_2132\958200182.py:29: SyntaxWarning: invalid escape sequence '\C'
  augmented_dataset = Cityscapes(root="E:\CityScapes", split='train', mode='fine', target_type='semantic', transform=augmented_transform, target_transform=augmented_target_transform)
c:\Users\Sven\anaconda3\envs\torch\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Comp

In [5]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [6]:
def mean_and_std(loader):
    mean = 0.
    std = 0.
    for images, _ in loader:
        batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)

    mean /= len(loader.dataset)
    std /= len(loader.dataset)
    return mean, std

#mean, std = mean_and_std(train_dataloader)

In [7]:

from model import Model
model = Model().cuda()


loss_fn = nn.CrossEntropyLoss(weight=weights, ignore_index=255)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)


In [8]:
def train(dataloader, model, loss_fn, optimizer, run):
    """
    Train a model for 1 epoch.

    Params:
    - dataloader:   dataset to train on.
    - model:        the model object to be trained.
    - loss_fn:      the loss function.
    - optimizer:    the desired optimization.
    """
    size = len(dataloader.dataset)
    model.train() #Set the model to train mode
    for batch, (IMG,SEGM) in enumerate(dataloader):
        IMG = IMG.to('cuda')
        SEGM  = (SEGM*255).long().squeeze()     #*255 because the id are normalized between 0-1
        SEGM = utils.map_id_to_train_id(SEGM).to('cuda')
        
        #predict
        pred = model(IMG)
        #Loss
        loss = loss_fn(pred, SEGM)
        

        #Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        #print loss during training
        loss, current = loss.item(), (batch + 1) * len(IMG)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        run.log({"train_loss": loss})
    
    return run
            


In [9]:
def test(dataloader, model, loss_fn):
    """
    Test a model.

    Params:
    - dataloader:   dataset to test on.
    - model:        the model object to be tested.
    - loss_fn:      the loss function.
    """
    num_batches = len(dataloader)
    model.eval() #model in eval mode
    test_loss = 0
    with torch.no_grad():
        for _, (IMG,SEGM) in enumerate(dataloader):
            IMG = IMG.to('cuda')
            SEGM  = (SEGM*255).long().squeeze()     #*255 because the id are normalized between 0-1
            SEGM = utils.map_id_to_train_id(SEGM).to('cuda')

            pred = model(IMG)
            test_loss += loss_fn(pred, SEGM).item()
            
    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")
    return test_loss
    


In [10]:



run = wandb.init(
    # Set the project where this run will be logged
    project="CS_challenge", name="1.8",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": {lr_rate},
        "epochs": {epochs},
        "Momentum": {momentum},
        "Batch_size": {batch_size},
        "model version": 0.0,
        "subset size [%]": 100,
        "resize": {subsize},
        "Normalized": False,
        "Optimizer": 'Adam',
        "weighted classes": True,
    },
)




for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    run = train(train_dataloader, model, loss_fn, optimizer, run)
    test_loss = test(val_dataloader, model, loss_fn)
    run.log({"Test_loss": test_loss, "Epoch": t})
print("Done!")


Epoch 1
-------------------------------
loss: 3.893908  [   64/ 4760]
loss: 3.390970  [  128/ 4760]
loss: 3.091957  [  192/ 4760]
loss: 2.915618  [  256/ 4760]
loss: 2.831128  [  320/ 4760]
loss: 2.516648  [  384/ 4760]
loss: 2.464610  [  448/ 4760]
loss: 2.217815  [  512/ 4760]
loss: 2.189900  [  576/ 4760]
loss: 2.147421  [  640/ 4760]
loss: 1.906577  [  704/ 4760]
loss: 1.846121  [  768/ 4760]
loss: 1.852178  [  832/ 4760]
loss: 1.810483  [  896/ 4760]
loss: 1.707912  [  960/ 4760]
loss: 1.657291  [ 1024/ 4760]
loss: 1.579134  [ 1088/ 4760]
loss: 1.623006  [ 1152/ 4760]
loss: 1.465245  [ 1216/ 4760]
loss: 1.451856  [ 1280/ 4760]
loss: 1.398741  [ 1344/ 4760]
loss: 1.469832  [ 1408/ 4760]
loss: 1.426301  [ 1472/ 4760]
loss: 1.446106  [ 1536/ 4760]
loss: 1.356265  [ 1600/ 4760]
loss: 1.285599  [ 1664/ 4760]
loss: 1.275582  [ 1728/ 4760]
loss: 1.247824  [ 1792/ 4760]
loss: 1.278619  [ 1856/ 4760]
loss: 1.271701  [ 1920/ 4760]
loss: 1.297851  [ 1984/ 4760]
loss: 1.281780  [ 2048/ 4760]


In [11]:
run.finish()

Epoch,▁▃▅▆█
Test_loss,█▃▃▁▁
train_loss,█▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁
Epoch,4
Test_loss,0.53167
train_loss,0.49469


In [12]:
torch.save(model.state_dict(), "model.pth")